# Compare ctmedit

> "Don't remember what I was doing here"

- branch: master
- hidden: true
- categories: [ctmedit, reverb-asr, hsi]

In [1]:
import pocketsphinx

In [4]:
textgrids="/Users/joregan/Playing/hsi/audio/whisperx-textgrids/"
ctmdir="/Users/joregan/Playing/hsi/ctc_prefix_beam_search/"
ctmeditdir="/Users/joregan/Playing/hsi/whisper_reverb_ctmedit/"

In [6]:
from pathlib import Path

def read_ctm(filename):
    if type(filename) is Path:
        filename = str(filename)

    ctmlines = []
    with open(filename) as inf:
        for line in inf.readlines():
            line = line.strip()
            ctmlines.append(line.split(" "))
    return ctmlines

In [5]:
from praatio import textgrid

def read_textgrid(filename, tiername="words"):
    if type(filename) is Path:
        filename = str(filename)
    tg = textgrid.openTextgrid(filename, False)

    utterances = []
    tier = tg.getTier(tiername)
    for entry in tier.entries:
        text = entry.label.strip()
        utterances.append((entry.start, entry.end, text))
    return utterances

In [8]:
tgpath = Path(textgrids)
ctmpath = Path(ctmdir)
ctmeditpath = Path(ctmeditdir)

In [9]:
EG = "hsi_5_0718_210_001_main"
CTMEDATA = read_ctm(ctmeditpath / f"{EG}.ctmedit")
TGDATA = read_textgrid(tgpath / f"{EG}.TextGrid")

In [18]:
def clean_word(word):
    return word.lower().strip(".,;:!?")

def map_words_to_tg(tgdata):
    mappings = []
    for num, item in enumerate(tgdata):
        for word in item[2].split(" "):
            clean = clean_word(word)
            mappings.append([clean, word, num])
    return mappings

In [20]:
def ctm_excerpt(ctmdata, start, end, fluff=0.6):
    if type(start) is str:
        start = float(start)
    if type(end) is str:
        end = float(end)
    
    excerpt = []
    for ctmline in ctmdata:
        cstart = float(ctmline[2])
        cdur = float(ctmline[3])
        cend = cstart + cdur
        if cstart < (start - fluff):
            continue
        if cend > (end + fluff):
            break
        excerpt.append(ctmline)
    return excerpt

In [33]:
def index_ctmdata_by_start(ctmdata):
    return {float(x[2]): n for n, x in enumerate(ctmdata)}

In [ ]:
index_ctmdata_by_start(CTMEDATA)

In [36]:
ctm_excerpt(CTMEDATA, 57.734, 60.576)

[['hsi_5_0718_210_001_main.wav', '1', '57.33', '0.1', 'i', '0.00', '-', 'ins'],
 ['hsi_5_0718_210_001_main.wav', '1', '57.57', '0.1', 'i', '0.00', 'i', 'cor'],
 ['hsi_5_0718_210_001_main.wav',
  '1',
  '57.85',
  '0.1',
  'really',
  '0.00',
  'really',
  'cor'],
 ['hsi_5_0718_210_001_main.wav',
  '1',
  '58.01',
  '0.1',
  'think',
  '0.00',
  'think',
  'cor'],
 ['hsi_5_0718_210_001_main.wav',
  '1',
  '58.17',
  '0.26',
  "i've",
  '0.00',
  "i've",
  'cor'],
 ['hsi_5_0718_210_001_main.wav',
  '1',
  '58.53',
  '0.1',
  'made',
  '0.00',
  'made',
  'cor'],
 ['hsi_5_0718_210_001_main.wav',
  '1',
  '58.77',
  '0.1',
  'the',
  '0.00',
  'the',
  'cor'],
 ['hsi_5_0718_210_001_main.wav',
  '1',
  '58.93',
  '0.1',
  'best',
  '0.00',
  'best',
  'cor'],
 ['hsi_5_0718_210_001_main.wav',
  '1',
  '59.03',
  '0.0',
  '-',
  '1.0',
  'out',
  'del'],
 ['hsi_5_0718_210_001_main.wav',
  '1',
  '59.37',
  '0.1',
  'outta',
  '0.00',
  'of',
  'sub'],
 ['hsi_5_0718_210_001_main.wav',
  '1',
 

In [42]:
def namethis(ctmedit, start, end, consumed_from=0):
    index = index_ctmdata_by_start(ctmedit)
    excerpt = ctm_excerpt(ctmedit, start, end)
    ex_start = float(excerpt[0][2])
    ex_start_ptr = index[ex_start]
    # if (ex_start_ptr - 1) > consumed_from:
    #     # extend here?
    #     pass
    prev_start = float(CTMEDATA[ex_start_ptr - 1][2])
    print(prev_start, ex_start, CTMEDATA[ex_start_ptr - 1])

In [44]:
namethis(CTMEDATA, 57.734, 60.576)

56.91 57.33 ['hsi_5_0718_210_001_main.wav', '1', '56.91', '0.04', 'think', '0.00', 'think', 'cor']


In [23]:
TGDATA

[(7.489, 7.709, 'Yeah.'),
 (7.729, 8.19, 'Yeah.'),
 (8.21, 8.49, 'Yes.'),
 (40.339, 41.12, 'Oh, that was nice.'),
 (41.461, 42.622, 'Nice idea to have that.'),
 (44.164, 45.025, 'Oh, I like this place.'),
 (55.393, 57.114, 'Yeah, so what do you think?'),
 (57.734,
  60.576,
  "I really think I've made the best out of this place actually."),
 (62.837, 63.038, 'Sorry?'),
 (63.918,
  72.263,
  "Yeah, it does and I really like what I did with the flowers up there because it's like the hanging gardens of Babylon."),
 (73.604, 77.026, 'I would like to have more of course, but I think in time'),
 (78.106, 82.527, 'They will hang down like a waterfall of green leaves.'),
 (83.368,
  88.609,
  "That's my intention anyway because I'm really into plants."),
 (90.149,
  98.892,
  "What's nice with these plants too, that's called the tongue of mother-in-law, the one by the window in Swedish."),
 (100.792, 103.194, "Because it's called Dragon Tongue."),
 (104.174,
  110.498,
  'And your mother-in-la

In [45]:
def partition_ctm(ctmdata, amount=1.0):
    segments = []
    current = []

    prev = 0.0
    for ctmline in ctmdata:
        start = float(ctmline[2])
        end = float(ctmline[3]) + start
        if prev > 0.0 and (end - prev) > amount:
            segments.append(current)
            current = []
        current.append(ctmline)
    segments.append(current)
    return segments

In [47]:
partitioned = partition_ctm(CTMEDATA)
for partition in partitioned:
    edit_type = set()
    for ctmline in partition:
        edit_type.add(ctmline[-1])
    if len(edit_type) == 1:
        print(partition)

In [49]:
partitioned[0][-1]

['hsi_5_0718_210_001_main.wav',
 '1',
 '734.04',
 '0.1',
 'yeah',
 '0.00',
 'yeah',
 'cor']